# Detecting Credit card fraud using Anomaly Detection algorithms

### The key idea is that instead of using the supervised learning models, we will build a probability of features for the normal transactions. Use the fradulent cases to set a threshold on what we call anomaly. 

Author: Sushant N. More

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal

Importing data

In [4]:
df = pd.read_csv('./creditcard.csv')

In [5]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [6]:
df.shape

(284807, 31)

We already did a detailed data exploration in the CreditCardFraudDetection notebook. 

Here are some of the points we remind ourselves.

1. The data is highly skewed. Out of 284,807 examples only 0.17% are fradulent. 

2. We noticed and reasoned that the time column doesn't have any effect on the transaction being fradulent or not. We drop it. 

3. We do the mean normalization of the amount column to make it amenable to ML techniques. 

4. We had seen that the probability density for features V1 through V28 roughly seems Gaussian

In the present case, the workflow is as follows: 

a) We repeat the steps 2 and 3 above.

b) We separate the normal transactions and do a probability density estimation. 

c) Use cross-validation to figure out the threshold. 

d) Finally test on the test set!

In [8]:
df['normAmount'] = StandardScaler().fit_transform(df['Amount'].values.reshape(-1,1))

In [9]:
df.columns

Index([u'Time', u'V1', u'V2', u'V3', u'V4', u'V5', u'V6', u'V7', u'V8', u'V9',
       u'V10', u'V11', u'V12', u'V13', u'V14', u'V15', u'V16', u'V17', u'V18',
       u'V19', u'V20', u'V21', u'V22', u'V23', u'V24', u'V25', u'V26', u'V27',
       u'V28', u'Amount', u'Class', u'normAmount'],
      dtype='object')

In [10]:
dfMod = df.drop(['Time', 'Amount'], axis = 1)

In [11]:
dfMod.columns

Index([u'V1', u'V2', u'V3', u'V4', u'V5', u'V6', u'V7', u'V8', u'V9', u'V10',
       u'V11', u'V12', u'V13', u'V14', u'V15', u'V16', u'V17', u'V18', u'V19',
       u'V20', u'V21', u'V22', u'V23', u'V24', u'V25', u'V26', u'V27', u'V28',
       u'Class', u'normAmount'],
      dtype='object')

In [12]:
dfMod.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


Let's separate the normal transactions from the fradulent

In [13]:
FraudTransaction = dfMod[dfMod['Class'] == 1]
NormalTransaction = dfMod[dfMod['Class'] == 0]

Note in this case, our training set consists of all normal examples. 
Let's split as follows:

Training set: 60% of the normal data

Cross validation set: 20% of normal data, 50% of fradulent data. 

Training set: 20% of normal data, 50% of fradulent data.

Before making this split, we randomly shuffle the data using the sample method of pandas and then apply np.split

In [15]:
len(NormalTransaction)

284315

In [16]:
NormalTransaction.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0,0.244964
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,0,-0.342475
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,0,1.160686
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0,0.140534
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,0,-0.073403


In [17]:
NormalTransaction.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            284797, 284798, 284799, 284800, 284801, 284802, 284803, 284804,
            284805, 284806],
           dtype='int64', length=284315)

In [18]:
NormalTransactionShuffled = NormalTransaction.sample(frac = 1)

In [19]:
NormalTransactionShuffled.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
227143,2.008021,-0.210928,-3.134833,-0.343703,2.597961,3.286339,-0.555582,0.832574,0.677253,-0.655852,...,-0.031826,0.094989,0.072766,0.631860,0.123913,0.659120,-0.022794,-0.035894,0,-0.313249
111151,1.178239,-0.112266,-0.528712,0.055660,-0.067912,-1.110653,0.532842,-0.377442,-0.088516,-0.116822,...,-0.027329,-0.272626,-0.226306,-0.034026,0.574890,1.092502,-0.133263,0.001482,0,0.046379
274188,-0.559310,0.252248,0.718171,-0.061253,1.167424,-0.583950,0.425633,0.035439,-0.653711,-0.226866,...,-0.137535,-0.517707,-0.022232,-0.462955,-0.407223,0.374007,0.067637,0.136836,0,-0.349671
36596,-0.707835,1.119971,1.288918,0.942082,-0.084526,-1.000469,0.756299,-0.238079,-0.299020,0.230965,...,0.138632,0.536785,-0.186578,0.713265,-0.390907,-0.434221,-0.539133,-0.359817,0,-0.311529
123354,-1.386725,1.067763,1.822499,-0.682421,-0.377616,-0.680118,0.282821,0.430088,0.150079,-0.372859,...,0.060073,0.011975,-0.091141,0.443813,-0.236341,-0.925158,-0.202221,-0.115952,0,-0.325283


In [20]:
NormalTransactionShuffled.index

Int64Index([227143, 111151, 274188,  36596, 123354, 188769, 143654, 271800,
            205058, 148641,
            ...
            270192, 169599, 223292, 146892, 284442,  62393, 101240, 116059,
             21779, 193586],
           dtype='int64', length=284315)

In [23]:
NormalTrain, NormalValidate, NormalFraud = np.split(NormalTransactionShuffled, \
                                                    [int(0.6 * len(NormalTransactionShuffled)), \
                                                    int(0.8 * len(NormalTransactionShuffled)) 
                                                    ])

In [24]:
NormalTrain.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
227143,2.008021,-0.210928,-3.134833,-0.343703,2.597961,3.286339,-0.555582,0.832574,0.677253,-0.655852,...,-0.031826,0.094989,0.072766,0.631860,0.123913,0.659120,-0.022794,-0.035894,0,-0.313249
111151,1.178239,-0.112266,-0.528712,0.055660,-0.067912,-1.110653,0.532842,-0.377442,-0.088516,-0.116822,...,-0.027329,-0.272626,-0.226306,-0.034026,0.574890,1.092502,-0.133263,0.001482,0,0.046379
274188,-0.559310,0.252248,0.718171,-0.061253,1.167424,-0.583950,0.425633,0.035439,-0.653711,-0.226866,...,-0.137535,-0.517707,-0.022232,-0.462955,-0.407223,0.374007,0.067637,0.136836,0,-0.349671
36596,-0.707835,1.119971,1.288918,0.942082,-0.084526,-1.000469,0.756299,-0.238079,-0.299020,0.230965,...,0.138632,0.536785,-0.186578,0.713265,-0.390907,-0.434221,-0.539133,-0.359817,0,-0.311529
123354,-1.386725,1.067763,1.822499,-0.682421,-0.377616,-0.680118,0.282821,0.430088,0.150079,-0.372859,...,0.060073,0.011975,-0.091141,0.443813,-0.236341,-0.925158,-0.202221,-0.115952,0,-0.325283


In [25]:
len(NormalTrain) + len(NormalValidate) + len(NormalFraud) 

284315

In [28]:
len(NormalTransaction)

284315

For the Fraud transactions, we can conveniently use the test_train split function from the sci-kit learn

In [29]:
FraudValidate, FraudTest = train_test_split(FraudTransaction, test_size = 0.5)

In [30]:
FraudValidate.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Class,normAmount
252774,-1.201398,4.864535,-8.328823,7.652399,-0.167445,-2.767695,-3.176421,1.623279,-4.367228,-5.533443,...,0.532320,-0.556913,0.192444,-0.698588,0.025003,0.514968,0.378105,-0.053133,1,-0.350151
46918,-4.423508,1.648048,-6.934388,4.894601,-5.078131,0.010849,-3.409096,1.409291,-3.260672,-7.781353,...,0.698359,0.487478,1.228698,-0.535217,0.388278,-0.009466,2.300164,0.081231,1,2.237530
263080,2.132386,0.705608,-3.530759,0.514779,1.527175,-1.716268,1.132791,-0.574214,0.128904,-1.000805,...,0.163739,0.703910,-0.245076,0.460049,0.920281,-0.216586,-0.026219,-0.025001,1,-0.349231
9035,-2.589617,7.016714,-13.705407,10.343228,-2.954461,-3.055116,-9.301289,3.349573,-5.654212,-11.853867,...,1.887738,0.333998,0.287659,-1.186406,-0.690273,0.631704,1.934221,0.789687,1,-0.349231
6899,-2.661802,5.856393,-7.653616,6.379742,-0.060712,-3.131550,-3.103570,1.778492,-3.831154,-7.191604,...,0.734775,-0.435901,-0.384766,-0.286016,1.007934,0.413196,0.280284,0.303937,1,-0.349231


In [31]:
len(FraudValidate)

246

In [32]:
len(FraudTest)

246

In [33]:
len(FraudTransaction)

492

Great, now let's model a probability distribution based on the normal examples in the training set. 

In [45]:
mu = NormalTrain.drop('Class', axis = 1).mean(axis = 0).values

In [46]:
mu

array([ 0.01454376, -0.00489052,  0.01024795, -0.00760175,  0.00357395,
        0.00314653,  0.00710655,  0.00067526,  0.00548902,  0.00793077,
       -0.00931862,  0.01185401,  0.00201089,  0.01150641, -0.00016354,
        0.0053953 ,  0.01247777,  0.00328759, -0.00268081, -0.00216582,
       -0.00294412,  0.0003498 , -0.00194123, -0.00103484,  0.00110445,
        0.00017356, -0.00153843,  0.00016741, -0.00239629])

In [51]:
Sigma = NormalTrain.drop('Class', axis = 1).cov().values

In [52]:
Sigma

array([[  3.72383943e+00,   4.98056772e-02,  -1.20253716e-01,
          6.47032573e-02,  -6.69683794e-02,  -2.03540491e-02,
         -1.19218445e-01,  -1.46868367e-02,  -4.18952552e-02,
         -8.35177645e-02,   4.34310395e-02,  -8.24347776e-02,
         -4.35948797e-04,  -7.04988450e-02,  -1.98384677e-03,
         -5.69897335e-02,  -1.07574914e-01,  -3.90558920e-02,
          1.32399132e-02,   1.33182573e-02,   2.31712203e-03,
         -8.34174264e-04,   2.19031996e-02,   9.22886948e-04,
          5.45562432e-03,   1.54939939e-03,   1.77006833e-02,
         -2.31847749e-02,  -4.31854759e-01],
       [  4.98056772e-02,   2.64486642e+00,   8.23263752e-02,
         -3.13171963e-02,   5.35054899e-02,   5.53683809e-03,
          7.68421477e-02,   8.38621869e-03,   2.23048775e-02,
          4.60994291e-02,  -3.51916284e-02,   5.70897630e-02,
         -2.42998416e-03,   6.37128938e-02,   5.16634028e-03,
          4.39446559e-02,   7.42052702e-02,   2.91739261e-02,
         -2.19650501e-03,

In [53]:
Sigma.shape

(29, 29)

In [54]:
model = multivariate_normal(cov=Sigma, mean=mu)

In [55]:
model